In [1]:
import os, time, shutil, tarfile
from minio import Minio
from minio.error import ResponseError
from tqdm import tnrange

times = 100
result = []
testList = [4, 8, 16, 32, 64, 128]

Gen test file

In [2]:
minioClient = Minio('140.124.181.77:32043',
                    access_key='wei840222',
                    secret_key='j04rmp4jo3',
                    secure=False)

for size in testList:
    with open('file.txt', 'w') as f:
        for i in range(size*1000*1000):
            f.write('a')
            
    try:
        minioClient.make_bucket('size-'+str(size)+'mb', location="us-east-1")
    except ResponseError as err:
        print(err)
    
    runTimeList = []
    for ti in tnrange(times, desc='times'):
        try:
            start = time.time()
            minioClient.fput_object('size-'+str(size)+'mb', str(ti), 'file.txt')
            end = time.time()
            runTimeList.append(end-start)
        except ResponseError as err:
            print(err)
            runTimeList.append(-1)
        time.sleep(5)

    result.append(runTimeList)
    try:
        os.remove('file.txt')
    except OSError as e:
        print ("Error: %s - %s." % (e.filename, e.strerror))

output result to csv

In [3]:
with open('minio/longhorn_upload_test_delay5.csv', 'w') as f:
    f.write(str([str(t)+'-MB' for t in testList]).replace("'",'')[1:-1]+'\n')
    for i in range(times):
        for j in range(len(result)):
            f.write(str(result[j][i]))
            if j != len(result) -1:
                f.write(', ')
        f.write('\n')

Download test

In [4]:
result = []
for size in testList:
    runTimeList = []
    for ti in tnrange(times, desc='times'):
        try:
            start = time.time()
            minioClient.fget_object('size-'+str(size)+'mbb', str(ti), 'file.txt')
            end = time.time()
            runTimeList.append(end-start)
        except ResponseError as err:
            print(err)
            runTimeList.append(-1)
        time.sleep(5)
        try:
            os.remove('file.txt')
        except OSError as e:
            print ("Error: %s - %s." % (e.filename, e.strerror))
    result.append(runTimeList)

NoSuchBucket: NoSuchBucket: message: The specified bucket does not exist.

In [ ]:
with open('minio/longhorn_download_test_delay5.csv', 'w') as f:
    f.write(str([str(t)+'-MB' for t in testList]).replace("'",'')[1:-1]+'\n')
    for i in range(times):
        for j in range(len(result)):
            f.write(str(result[j][i]))
            if j != len(result) -1:
                f.write(', ')
        f.write('\n')